---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
hand = open('../text_files/university_towns.txt')
UTowns = pd.DataFrame(columns=('State', 'RegionName'))
i = 0
for line in hand:
    line = line.rstrip()
    if 'edit' in line:
        State = line.replace('[edit]', "")
    else:
        RegionName = re.sub(r' \(.*', "", line)
        UTowns.loc[i] = [State, RegionName]
        i += 1

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    hand = open('../text_files/university_towns.txt')
    UTowns = pd.DataFrame(columns=('State', 'RegionName'))
    i = 0
    for line in hand:
        line = line.rstrip()
        if 'edit' in line:
            State = line.replace('[edit]', "")
        else:
            RegionName = re.sub(r' \(.*', "", line)
            UTowns.loc[i] = [State, RegionName]
            i += 1
    return UTowns
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [98]:
GDP = pd.read_excel('../datasets/gdplev.xls', skiprows = 224, header = None)
GDP = GDP[[4, 5, 6]]
GDP.columns = ['Quarter', 'GDP current dollars', 'GDP 2009 dollars']
i = 2
while i < len(GDP['Quarter']):
    if GDP.loc[i]['GDP current dollars'] < GDP.loc[i-1]['GDP current dollars'] and GDP.loc[i-1]['GDP current dollars'] < GDP.loc[i-2]['GDP current dollars']:
        recession_start = GDP.loc[i-2]['Quarter']
        recession_start_index = i-2 
        break
    i += 1
recession_start

'2008q3'

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('../datasets/gdplev.xls', skiprows = 224, header = None)
    GDP = GDP[[4, 5, 6]]
    GDP.columns = ['Quarter', 'GDP current dollars', 'GDP 2009 dollars']
    i = 2
    while i < len(GDP['Quarter']):
        if GDP.loc[i]['GDP current dollars'] < GDP.loc[i-1]['GDP current dollars'] and GDP.loc[i-1]['GDP current dollars'] < GDP.loc[i-2]['GDP current dollars']:
            recession_start = GDP.loc[i-2]['Quarter']
            recession_start_index = i-2 
            break
        i += 1
    return recession_start
get_recession_start()

'2008q3'

In [97]:
i = recession_start_index + 2
while i < len(GDP['Quarter']):
    if GDP.loc[i]['GDP current dollars'] > GDP.loc[i-1]['GDP current dollars'] and GDP.loc[i-1]['GDP current dollars'] > GDP.loc[i-2]['GDP current dollars']:
        recession_end = GDP.loc[i]['Quarter']
        recession_end_index = i 
        break
    i += 1
recession_end

'2009q4'

In [27]:
GDP = pd.read_excel('../datasets/gdplev.xls', skiprows = 224, header = None)
GDP = GDP[[4, 5, 6]]
GDP.columns = ['Quarter', 'GDP current dollars', 'GDP 2009 dollars']
a = GDP.index[GDP['Quarter'] == get_recession_start()].tolist()[0]
a

30

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('../datasets/gdplev.xls', skiprows = 224, header = None)
    GDP = GDP[[4, 5, 6]]
    GDP.columns = ['Quarter', 'GDP current dollars', 'GDP 2009 dollars']
    recession_start_index = GDP.index[GDP['Quarter'] == get_recession_start()].tolist()[0]
    i = recession_start_index + 2
    while i < len(GDP['Quarter']):
        if GDP.loc[i]['GDP current dollars'] > GDP.loc[i-1]['GDP current dollars'] and GDP.loc[i-1]['GDP current dollars'] > GDP.loc[i-2]['GDP current dollars']:
            recession_end = GDP.loc[i]['Quarter']
            recession_end_index = i 
            break
        i += 1    
    return recession_end
get_recession_end()

'2009q4'

In [106]:
recession_bottom = None
for i in range(recession_start_index, recession_end_index):
    if recession_bottom == None:
        recession_bottom_quarter = GDP.loc[i]['Quarter']
        recession_bottom = GDP.loc[i]['GDP current dollars']
    elif GDP.loc[i]['GDP current dollars'] < recession_bottom:
        recession_bottom_quarter = GDP.loc[i]['Quarter']
        recession_bottom = GDP.loc[i]['GDP current dollars']


'2009q2'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('../datasets/gdplev.xls', skiprows = 224, header = None)
    GDP = GDP[[4, 5, 6]]
    GDP.columns = ['Quarter', 'GDP current dollars', 'GDP 2009 dollars']
    
    recession_start_index = GDP.index[GDP['Quarter'] == get_recession_start()].tolist()[0]
    recession_end_index = GDP.index[GDP['Quarter'] == get_recession_end()].tolist()[0]
    recession_bottom = None
    for i in range(recession_start_index, recession_end_index):
        if recession_bottom == None:
            recession_bottom_quarter = GDP.loc[i]['Quarter']
            recession_bottom = GDP.loc[i]['GDP current dollars']
        elif GDP.loc[i]['GDP current dollars'] < recession_bottom:
            recession_bottom_quarter = GDP.loc[i]['Quarter']
            recession_bottom = GDP.loc[i]['GDP current dollars']
    return recession_bottom_quarter
get_recession_bottom()

'2009q2'

In [15]:
Housing = pd.read_csv('../datasets/City_Zhvi_AllHomes.csv')
Housing = Housing.replace(to_replace = states).set_index(['State', 'RegionName']).drop(columns=['RegionID', 'Metro', 'CountyName', 'SizeRank'])
# Housing.groupby(pd.PeriodIndex(Housing.columns, freq='Q'), axis=1).mean()
Housing = Housing.groupby(pd.PeriodIndex(Housing.columns, freq='q'), axis=1).mean()
columns_to_drop = []
for i in range(0,15):    
    columns_to_drop.append(Housing.columns[i])
columns_to_drop
Housing = Housing.drop(columns=columns_to_drop)
Housing

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    Housing = pd.read_csv('../datasets/City_Zhvi_AllHomes.csv')
    Housing = Housing.replace(to_replace = states).set_index(['State', 'RegionName']).drop(columns=['RegionID', 'Metro', 'CountyName', 'SizeRank'])
    # Housing.groupby(pd.PeriodIndex(Housing.columns, freq='Q'), axis=1).mean()
    Housing = Housing.groupby(pd.PeriodIndex(Housing.columns, freq='Q'), axis=1).mean()
    columns_to_drop = []
    for i in range(0,15):    
        columns_to_drop.append(Housing.columns[i])
    Housing = Housing.drop(columns=columns_to_drop)
    return Housing
convert_housing_data_to_quarters()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [13]:
UTowns = get_list_of_university_towns()
recession_bottom = get_recession_bottom()
recession_start = get_recession_start()
Housing = convert_housing_data_to_quarters()

In [14]:
Housing['Delta'] = Housing[recession_bottom]-Housing[recession_start]
utown_merge = pd.merge(UTowns, Housing, how = 'inner', left_on = ['State', 'RegionName'], right_on = ['State', 'RegionName'])
utown_merge['University'] = True
all_town = pd.merge(utown_merge, Housing, how = 'outer', on = ['State', 'RegionName', 'Delta']).fillna(False)
non_utown = all_town[all_town['University']==False]
utown = all_town[all_town['University']==True]
# different = True if p < 0.01 else False
# ttest_ind(utown['Delta'], non_utown['Delta'])
all_town

,State,RegionName,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
0,Alabama,Montevallo,97000,96800,96533.3,98333.3,99466.7,101333,103200,101867,...,1.193667e+05,1.202667e+05,1.200000e+05,1.199000e+05,1.194667e+05,1.194000e+05,1.198333e+05,1.217000e+05,1.224000e+05,121500.0
1,Alabama,Tuscaloosa,104500,106033,108500,108767,109900,111367,110000,110700,...,1.379667e+05,1.395667e+05,1.427333e+05,1.450000e+05,1.447000e+05,1.442667e+05,1.450000e+05,1.403000e+05,1.380333e+05,138650.0
2,Alaska,Fairbanks,163200,165033,169300,172800,164433,157800,158200,154667,...,2.092333e+05,2.071000e+05,2.070000e+05,2.077667e+05,2.064667e+05,2.084333e+05,2.094667e+05,2.090667e+05,2.129333e+05,215850.0
3,Arizona,Flagstaff,161300,164100,165467,168267,172000,175633,178967,180800,...,2.909667e+05,2.950667e+05,2.959667e+05,2.976333e+05,3.028000e+05,3.073333e+05,3.100333e+05,3.122667e+05,3.184333e+05,322650.0
4,Arizona,Tempe,128100,129933,131100,132800,135633,137900,139700,141833,...,2.097667e+05,2.130667e+05,2.168667e+05,2.204667e+05,2.259000e+05,2.312000e+05,2.378000e+05,2.419000e+05,2.450000e+05,246950.0
5,Arizona,Tucson,101833,102967,104467,105667,107200,108767,110567,112800,...,1.424667e+05,1.434333e+05,1.442333e+05,1.441667e+05,1.451333e+05,1.466000e+05,1.481667e+05,1.495333e+05,1.511667e+05,152700.0
6,Arkansas,Conway,85866.7,87166.7,88366.7,88800,89333.3,90266.7,89466.7,89066.7,...,1.315333e+05,1.309667e+05,1.322000e+05,1.326333e+05,1.333667e+05,1.346000e+05,1.353667e+05,1.362000e+05,1.373333e+05,138300.0
7,Arkansas,Fayetteville,107933,110333,112567,114600,116933,119367,120167,119967,...,1.684333e+05,1.692333e+05,1.714000e+05,1.712333e+05,1.699000e+05,1.699667e+05,1.716333e+05,1.774000e+05,1.821333e+05,187950.0
8,California,Arcata,130033,140933,149200,150767,153233,151767,157067,173467,...,2.674333e+05,2.659667e+05,2.656000e+05,2.701667e+05,2.787333e+05,2.871667e+05,2.902333e+05,2.924667e+05,3.005000e+05,306200.0
9,California,Berkeley,395633,431033,464367,480867,496567,497667,483267,476233,...,8.010333e+05,8.161333e+05,8.471333e+05,8.861333e+05,9.256000e+05,9.579667e+05,9.696333e+05,9.872000e+05,9.916333e+05,987100.0


In [43]:
t, p = ttest_ind(utown['Delta'].dropna(), non_utown['Delta'].dropna())
different = True if p < 0.01 else False
if utown['Delta'].mean() < non_utown['Delta'].mean():
    better = 'university town'
else:
    better = 'non-university town'
different, p, better

(False, 0.011532873983089074, 'non-university town')

In [49]:
type(get_recession_bottom())

str

In [50]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    UTowns = get_list_of_university_towns()
    recession_bottom = get_recession_bottom()
    recession_start = get_recession_start()
    Housing = convert_housing_data_to_quarters()
    
    Housing['Delta'] = Housing[recession_bottom]-Housing[recession_start]
    utown_merge = pd.merge(UTowns, Housing, how = 'inner', left_on = ['State', 'RegionName'], right_on = ['State', 'RegionName'])
    utown_merge['University'] = True
    all_town = pd.merge(utown_merge, Housing, how = 'outer', on = ['State', 'RegionName', 'Delta']).fillna(False)
    non_utown = all_town[all_town['University']==False]
    utown = all_town[all_town['University']==True]
    
    t, p = ttest_ind(utown['Delta'].dropna(), non_utown['Delta'].dropna())
    different = True if p < 0.01 else False
    if utown['Delta'].mean() < non_utown['Delta'].mean():
        better = 'university town'
    else:
        better = 'non-university town'

    return different, p, better


In [135]:
UTowns = get_list_of_university_towns()
recession_bottom = get_recession_bottom()
recession_start = get_recession_start()
Housing = convert_housing_data_to_quarters()
# qrt_bfr_rec_start = recession_start - 1q

Housing['ratio'] = Housing['2008Q2'].div(Housing[recession_bottom])
prices = Housing['ratio']
prices = prices.reset_index()
utown_merge = pd.merge(UTowns, prices, how = 'inner', left_on = ['State', 'RegionName'], right_on = ['State', 'RegionName'])
utown_merge['University'] = True
all_town = pd.merge(utown_merge, prices, how = 'outer', on = ['State', 'RegionName', 'ratio'])
all_town['University'] = all_town['University'].fillna(False)
non_utown = all_town[all_town['University']==False]
utown = all_town[all_town['University']==True]
    
t, p = ttest_ind(utown['ratio'].dropna(), non_utown['ratio'].dropna())
different = True if p < 0.01 else False
if utown['ratio'].mean() < non_utown['ratio'].mean():
    better = 'university town'
else:
    better = 'non-university town'

In [138]:
different

True

In [131]:
unitowns = get_list_of_university_towns()
bottom = get_recession_bottom()
start = get_recession_start()
hdata = convert_housing_data_to_quarters()
bstart = hdata.columns[hdata.columns.get_loc(start) -1]
    
hdata['ratio'] = hdata[bstart].div(hdata[bottom])
hdata = hdata['ratio']
hdata = hdata.reset_index()
unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
unitowns_hdata['uni'] = True
hdata2 = pd.merge(hdata,unitowns_hdata,how='outer',on=['State','RegionName','ratio'])
hdata2['uni'] = hdata2['uni'].fillna(False)

ut = hdata2[hdata2['uni'] == True]
nut = hdata2[hdata2['uni'] == False]

t,p = ttest_ind(ut['ratio'].dropna(),nut['ratio'].dropna())
    
different = True if p < 0.01 else False

better = "non-university town" if ut['ratio'].mean() > nut['ratio'].mean() else "university town"

different, p, better
# bottom

(True, 0.002724063704761454, 'university town')

In [130]:
nut['ratio'].mean()

1.0751962209307955

In [51]:
run_ttest()

(False, 0.011532873983089074, 'non-university town')